## Evolutionary origin

Macaca - Mouse - Human

In [4]:
import os,re,glob
import pandas as pd
import numpy as np
from collections import Counter
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

species_dir = "/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction"
users_dir = "/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/v47"

GENOMEDIR = "/genomics/users/marta/genomes"
species=["mouse","macaca","platypus","chicken","opossum"]

## annotation file
annotation="/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/v47/human/newReference_Resconstructed/gencode.v47.gffcompare.TestisLiverBrain.annotation.sorted.1transcript.sorted.NOchr.gtf"
transcript_gene=pd.read_csv("/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/v47/human/newReference_Resconstructed/transID_geneID_isoforms_selected.1to1.csv")

## evoDir
evoDir=os.path.join(users_dir,"EvolutionaryOrigin_MMseqs")

def translate_dna_to_protein(dna_seq):
    return str(Seq(dna_seq).translate())

# Function to save DF in fasta format
def create_seqrecord_PROT(row):
    return SeqRecord(Seq(row['protein']), id=row['header'][:-4], description="")
def create_seqrecord_DNA(row):
    return SeqRecord(Seq(row['seq']), id=row['header'], description="")

species_and_years_dict = {"specie" : ["human","macaca","mouse","opossum","platypus","chicken"],
                          "age" : [0,25,90,180,200,310]}
species_and_years = pd.DataFrame(data=species_and_years_dict, index=[1,2,3,4,5,6])
species_and_years


,specie,age
1,human,0
2,macaca,25
3,mouse,90
4,opossum,180
5,platypus,200
6,chicken,310


In [5]:
# ## get repre sequences - both DNA and PROTEINS
# for s in species:
#     print(s)
#     riboseq_dir = os.path.join(users_dir,s,"RiboSeq/RiboQC_RiboNovel")

#     ## get all candidates
#     print("Reading all the candidate ORF sequences")
#     df = pd.read_csv(os.path.join(riboseq_dir,"Annotation/candidateORF.fa"), header=None, sep="\t")
#     ## convert to df
#     candidates_fasta = pd.DataFrame({'header':df[0].iloc[::2].values, 'seq':df[0].iloc[1::2].values})
#     candidates_fasta['header'] = candidates_fasta['header'].str[1:]
#     candidates_fasta = candidates_fasta[~candidates_fasta['header'].str.contains("iORF")]

#     all_repre_candidates = pd.DataFrame()
#     for sample in os.listdir(os.path.join(riboseq_dir,"RibORF")):
#         if sample.endswith("r1"):
#             print(sample)

#             repre = pd.read_csv(os.path.join(riboseq_dir,"RibORF",sample,"repre.valid.pred.pvalue.parameters.txt"), sep="\t")
#             repre_fasta = candidates_fasta[candidates_fasta['header'].isin(repre.orfID.values.tolist())]
#             repre_fasta['header'] = s + "_" +repre_fasta['header']

#             ## keep only the coding and the non-coding
#             repre_fasta = repre_fasta[repre_fasta['header'].str.contains("canonical")| repre_fasta['header'].str.contains("noncoding")]

#             repre_fasta['protein'] = repre_fasta['seq'].apply(translate_dna_to_protein)
#             all_repre_candidates = pd.concat([all_repre_candidates, repre_fasta])

#             ## translate to protein
#             print("Translating to protein")
#             out_proteins = os.path.join(riboseq_dir,"RibORF",sample,"repre.valid.pred.pvalue.parameters.PROTEIN.fa")
#             seq_records_proteins = repre_fasta.apply(create_seqrecord_PROT, axis=1).tolist()
#             SeqIO.write(seq_records_proteins, out_proteins, "fasta")

#             ## keep the dna sequences as well
#             print("Saving the original DNA ORF")
#             out_dna = os.path.join(riboseq_dir,"RibORF",sample,"repre.valid.pred.pvalue.parameters.DNA.fa")
#             seq_records_dna = repre_fasta.apply(create_seqrecord_DNA, axis=1).tolist()
#             SeqIO.write(seq_records_dna, out_dna, "fasta")

#             ## get only the nonredundant
#             print("Getting the non-redundant")
#             all_repre_candidates.drop_duplicates(inplace=True)
#             out_nonredundant_proteins = os.path.join(riboseq_dir,"RibORF")+"/"+s+"_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa"
#             seq_records_proteins = all_repre_candidates.apply(create_seqrecord_PROT, axis=1).tolist()
#             SeqIO.write(seq_records_proteins, out_nonredundant_proteins, "fasta")

## MMseqs2

Against all expressed in mouse, macaca, platypus, opossum in any tissue (non-redunant set)

#### TESTIS-EXPRESSED

In [6]:
## Testis-expressed ORFs human
TestisExpressed_translatedINtestis = pd.read_csv("/projects_eg/projects/marta/TestisRestricted_Microproteins_TSA/with_TranscriptomeReconstruction/Multimap_altORFs/Q1_TestisORFs/human/ribORF_humanTestis_in1.csv")
TestisExpressed_translatedINtestis

def create_seqrecord_PROThuman(row):
    return SeqRecord(Seq(row['ORFpep']), id=row['orfID'], description="")

out_nonredundant_proteins = "/projects_eg/projects/marta/TestisRestricted_Microproteins_TSA/with_TranscriptomeReconstruction/Multimap_altORFs/Q1_TestisORFs/human/ribORF_humanTestis_in1.fa"
seq_records_proteins = TestisExpressed_translatedINtestis.apply(create_seqrecord_PROThuman, axis=1).tolist()
SeqIO.write(seq_records_proteins, out_nonredundant_proteins, "fasta")

27251

Creating database of human (expressed in testis)

In [7]:
%%bash -s "$evoDir" "$users_dir"

# mkdir $1/MMseqsDB

module load Miniconda3/4.9.2
source /soft/system/software/Miniconda3/4.9.2/bin/activate MMseqs2

mkdir -p $1/MMseqsDB

mmseqs createdb /projects_eg/projects/marta/TestisRestricted_Microproteins_TSA/with_TranscriptomeReconstruction/Multimap_altORFs/Q1_TestisORFs/human/ribORF_humanTestis_in1.fa $1/MMseqsDB/ribORF_humanTestis_in1_DB


/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/v47/EvolutionaryOrigin_MMseqs/MMseqsDB/ribORF_humanTestis_in1_DB exists and will be overwritten
createdb /projects_eg/projects/marta/TestisRestricted_Microproteins_TSA/with_TranscriptomeReconstruction/Multimap_altORFs/Q1_TestisORFs/human/ribORF_humanTestis_in1.fa /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/v47/EvolutionaryOrigin_MMseqs/MMseqsDB/ribORF_humanTestis_in1_DB 

MMseqs Version:       	14.7e284
Database type         	0
Shuffle input database	true
Createdb mode         	0
Write lookup file     	1
Offset of numeric ids 	0
Compressed            	0
Verbosity             	3

Converting sequences
[==
Time for merging to ribORF_humanTestis_in1_DB_h: 0h 0m 0s 684ms
Time for merging to ribORF_humanTestis_in1_DB: 0h 0m 2s 772ms
Database type: Aminoacid
Time for processing: 0h 0m 15s 217ms


Creating a joint database with all the specie, everything translated, no matter the tissue

In [8]:
%%bash -s "$evoDir" "$species_dir"

module load Miniconda3/4.9.2
source /soft/system/software/Miniconda3/4.9.2/bin/activate MMseqs2

# species="platypus mouse macaca chicken"
# for s in $species; do
#     echo $s

#     ### create DB
#     mmseqs createdb $2/$s/RiboSeq/RiboQC_RiboNovel/RibORF/repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa $1/MMseqsDB/${s}_noredunantProts_DB

# done

mmseqs createdb $2/*/RiboSeq/RiboQC_RiboNovel*/RibORF/*_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa $1/MMseqsDB/speciesDB



/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/v47/EvolutionaryOrigin_MMseqs/MMseqsDB/speciesDB exists and will be overwritten
createdb /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/chicken/RiboSeq/RiboQC_RiboNovel/RibORF/chicken_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/macaca/RiboSeq/RiboQC_RiboNovel/RibORF/macaca_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/mouse/RiboSeq/RiboQC_RiboNovel/RibORF/mouse_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/opossum/RiboSeq/RiboQC_RiboNovel/RibORF/opossum_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/platypus/Ri

Creating a joint database with all the specie AND human for clustering, everything translated for the species, no matter the tissue, but only translated in 1 for testis

In [9]:
%%bash -s "$evoDir" "$species_dir"

module load Miniconda3/4.9.2
source /soft/system/software/Miniconda3/4.9.2/bin/activate MMseqs2

mmseqs createdb $2/*/RiboSeq/RiboQC_RiboNovel*/RibORF/*_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa /projects_eg/projects/marta/TestisRestricted_Microproteins_TSA/with_TranscriptomeReconstruction/Multimap_altORFs/Q1_TestisORFs/human/ribORF_humanTestis_in1.fa $1/MMseqsDB/species_and_humanDB



/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/v47/EvolutionaryOrigin_MMseqs/MMseqsDB/species_and_humanDB exists and will be overwritten
createdb /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/chicken/RiboSeq/RiboQC_RiboNovel/RibORF/chicken_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/macaca/RiboSeq/RiboQC_RiboNovel/RibORF/macaca_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/mouse/RiboSeq/RiboQC_RiboNovel/RibORF/mouse_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/opossum/RiboSeq/RiboQC_RiboNovel/RibORF/opossum_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/p

Searching

In [10]:
# %%bash -s "$evoDir" "$users_dir"

# module load Miniconda3/4.9.2
# source /soft/system/software/Miniconda3/4.9.2/bin/activate MMseqs2
# mkdir -p $1/Searching

# rm $1/Searching/resultDB*
# ## The alignment consists of two steps the prefilter and alignment. To run the search, type:
# mmseqs search -a -s 6 $1/MMseqsDB/ribORF_humanTestis_in1_DB $1/MMseqsDB/speciesDB $1/Searching/resultDB $1/tmp

# # Then, convert the result database into a BLAST tab formatted file (option -m 8 in legacy blast, -outfmt 6 in blast+):
# # Format alignment output	query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits

# mmseqs convertalis $1/MMseqsDB/ribORF_humanTestis_in1_DB $1/MMseqsDB/speciesDB $1/Searching/resultDB $1/Searching/resultDB.m8

Clustering

In [11]:
%%bash -s "$evoDir" "$users_dir"

module load Miniconda3/4.9.2
source /soft/system/software/Miniconda3/4.9.2/bin/activate MMseqs2

mkdir -p $1/Clustering

rm -r $1/Clustering/species_and_human_cluDB*

## cluster all species and human
mmseqs cluster -c 0.5 --min-seq-id 0.5 $1/MMseqsDB/species_and_humanDB $1/Clustering/species_and_human_cluDB $1/tmp
mmseqs createtsv $1/MMseqsDB/species_and_humanDB $1/MMseqsDB/species_and_humanDB $1/Clustering/species_and_human_cluDB $1/Clustering/species_and_human_cluDB.tsv

cluster -c 0.5 --min-seq-id 0.5 /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/v47/EvolutionaryOrigin_MMseqs/MMseqsDB/species_and_humanDB /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/v47/EvolutionaryOrigin_MMseqs/Clustering/species_and_human_cluDB /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/v47/EvolutionaryOrigin_MMseqs/tmp 

MMseqs Version:                     	14.7e284
Substitution matrix                 	aa:blosum62.out,nucl:nucleotide.out
Seed substitution matrix            	aa:VTML80.out,nucl:nucleotide.out
Sensitivity                         	4
k-mer length                        	0
k-score                             	seq:2147483647,prof:2147483647
Alphabet size                       	aa:21,nucl:5
Max sequence length                 	65535
Max results per query               	20
Split database                      	0
Split mode                          	2
Split memory limit

## Analysis

In [12]:
clustering = pd.read_csv(os.path.join(evoDir,"Clustering/species_and_human_cluDB.tsv"), header=None, sep="\t")
clustering = clustering[clustering[0].str.startswith("ENST") | clustering[0].str.startswith("TCONS")]
print("all human")
print(len(clustering))
clustering['transcript_id'] = clustering[0].str.split(":", expand=True)[0]
clustering['transcript_id'] = clustering["transcript_id"].str.split(".", expand=True)[0]

clustering = clustering.merge(transcript_gene, on="transcript_id")
print("all human after annotation")
print(len(clustering))
small_clust = clustering[["gene_name","gene_type",0,1]]
small_clust.groupby("gene_type").count()


all human
26375
all human after annotation
26375


,gene_name,0,1
gene_type,,,
lncRNA,2318,2318,2318
novel,130,130,130
processed_pseudogene,1557,1557,1557
protein_coding,22370,22370,22370


In [13]:
small_clust

,gene_name,gene_type,0,1
0,R3HCC1L,protein_coding,ENST00000298999.8:10:+|18|3393:305:350|ouORF|TTG,ENST00000298999.8:10:+|18|3393:305:350|ouORF|TTG
1,R3HCC1L,protein_coding,ENST00000298999.8:10:+|8|3393:157:268|uORF|GTG,ENST00000298999.8:10:+|8|3393:157:268|uORF|GTG
2,R3HCC1L,protein_coding,ENST00000298999.8:10:+|2|3393:78:282|uORF|CTG,ENST00000298999.8:10:+|2|3393:78:282|uORF|CTG
3,R3HCC1L,protein_coding,ENST00000298999.8:10:+|151|3393:3093:3348|dORF...,ENST00000298999.8:10:+|151|3393:3093:3348|dORF...
4,CHODL,protein_coding,ENST00000299295.7:21:+|58|2545:1089:1329|odORF...,ENST00000299295.7:21:+|58|2545:1089:1329|odORF...
...,...,...,...,...
26370,FOXRED2,protein_coding,ENST00000684868.1:22:-|6|5640:116:494|uORF|CTG,ENST00000684868.1:22:-|6|5640:116:494|uORF|CTG
26371,EXOSC8,protein_coding,ENST00000685624.1:13:+|32|4097:704:755|uORF|ATG,ENST00000685624.1:13:+|32|4097:704:755|uORF|ATG
26372,SNHG33,lncRNA,ENST00000686765.2:3:+|2|1688:28:175|noncoding|ATG,ENST00000686765.2:3:+|2|1688:28:175|noncoding|ATG
26373,LINC01871,lncRNA,ENST00000690653.3:2:+|23|2292:494:677|noncodin...,ENST00000690653.3:2:+|23|2292:494:677|noncodin...


In [14]:
## non-coding
noncoding = clustering[clustering[0].str.contains("noncoding")]
noncoding.to_csv(os.path.join(evoDir,"Clustering/noncodingHuman_speciesDB.tsv"), header=None, index=None, sep="\t")
print("noncoding all")
print(len(noncoding))
print(len(set(noncoding[0].values.tolist())))

## altORFs
altORFs = clustering[clustering[0].str.contains("ORF")]
altORFs.to_csv(os.path.join(evoDir,"Clustering/altORFsHuman_speciesDB.tsv"), header=None, index=None, sep="\t")
print("altORFs all")
print(len(altORFs))
print(len(set(altORFs[0].values.tolist())))

## canonical
canonical = clustering[clustering[0].str.contains("canonical")]
print("Canonical all")
print(len(canonical))
print(len(set(canonical[0].values.tolist())))
canonical.to_csv(os.path.join(evoDir,"Clustering/canonicalHuman_speciesDB.tsv"), header=None, index=None, sep="\t")

noncoding all
4005
3308
altORFs all
7827
7516
Canonical all
14543
2909


#### NONCODING

In [15]:
# if column 1 starts with a specie, it is conserved
noncoding_conserved = noncoding[~noncoding[1].str.startswith("ENST")]
noncoding_conserved = noncoding_conserved[~noncoding_conserved[1].str.startswith("TCONS")]
noncoding_conserved.to_csv(os.path.join(evoDir,"Clustering/noncodingHuman_Conserved.tsv"), header=None, index=None, sep="\t")
print("Conserved noncoding human ORFs")
print(len(set(noncoding_conserved[0].values.tolist())))
noncoding_conserved = noncoding_conserved[['gene_name','gene_type',0,1]]
noncoding_conserved.columns = ['gene_name','gene_type','orfID',1]

# if column 1 starts with a human gene, it is not conserved
noncoding_NonConserved = noncoding[noncoding[1].str.startswith("ENST") | noncoding[1].str.startswith("TCONS")]
noncoding_NonConserved.to_csv(os.path.join(evoDir,"Clustering/noncodingHuman_NonConserved.tsv"), header=None, index=None, sep="\t")
print("Non-Conserved noncoding human ORFs")
print(len(set(noncoding_NonConserved[0].values.tolist())))
noncoding_NonConserved = noncoding_NonConserved[['gene_name','gene_type',0,1]]
noncoding_NonConserved.columns = ['gene_name','gene_type','orfID',1]
noncoding_NonConserved

Conserved noncoding human ORFs
176
Non-Conserved noncoding human ORFs
3308


,gene_name,gene_type,orfID,1
190,ENSG00000260422,lncRNA,ENST00000564697.1:6:-|17|1639:356:590|noncodin...,ENST00000564697.1:6:-|17|1639:356:590|noncodin...
193,ENSG00000290665,lncRNA,ENST00000567226.2:15:-|6|2266:104:320|noncodin...,ENST00000567226.2:15:-|6|2266:104:320|noncodin...
210,LINC02864,lncRNA,ENST00000581862.7:18:-|12|3689:297:567|noncodi...,ENST00000581862.7:18:-|12|3689:297:567|noncodi...
212,RDM1P5,lncRNA,ENST00000583447.6:17:+|10|2309:144:324|noncodi...,ENST00000583447.6:17:+|10|2309:144:324|noncodi...
241,SINHCAFP2,processed_pseudogene,ENST00000591086.1:18:-|2|664:24:237|noncoding|ATG,ENST00000591086.1:18:-|2|664:24:237|noncoding|ATG
...,...,...,...,...
26360,DUSP5-DT,lncRNA,ENST00000666621.1:10:-|25|3011:473:665|noncodi...,ENST00000666621.1:10:-|25|3011:473:665|noncodi...
26361,MAP4K3-DT,lncRNA,ENST00000671403.1:2:+|11|4190:235:391|noncodin...,ENST00000671403.1:2:+|11|4190:235:391|noncodin...
26372,SNHG33,lncRNA,ENST00000686765.2:3:+|2|1688:28:175|noncoding|ATG,ENST00000686765.2:3:+|2|1688:28:175|noncoding|ATG
26373,LINC01871,lncRNA,ENST00000690653.3:2:+|23|2292:494:677|noncodin...,ENST00000690653.3:2:+|23|2292:494:677|noncodin...


In [16]:
## conserved non-coding
noncoding_conserved_to_matrix = noncoding_conserved
print(len(set(noncoding_conserved_to_matrix.orfID.values.tolist())))
# Assuming noncoding_conserved_to_matrix is already defined
noncoding_conserved_to_matrix['specie'] = noncoding_conserved_to_matrix[1].str.split("_", expand=True)[0]
noncoding_conserved_to_matrix = noncoding_conserved_to_matrix[['orfID', 'specie']]
noncoding_conserved_to_matrix.groupby('specie').count()
noncoding_conserved_to_matrix['presence'] = 1
## control
print(len(set(noncoding_conserved_to_matrix.orfID.values.tolist())))

# Pivoting the data without creating an 'id', and keeping 'orfID' as the unique identifier
noncoding_conserved_matrix = noncoding_conserved_to_matrix.pivot_table(index='orfID', columns='specie', values='presence', aggfunc='max')

# Fill NaN values with 0
noncoding_conserved_matrix = noncoding_conserved_matrix.fillna(0)

# Reset index so 'orfID' becomes a column instead of an index
noncoding_conserved_matrix = noncoding_conserved_matrix.reset_index()

# Reorder columns for the final matrix
noncoding_conserved_matrix_ordered = noncoding_conserved_matrix[["orfID", "macaca", "mouse", "opossum", "platypus", "chicken"]]

# Calculate sum across species columns to identify how many species are present
noncoding_conserved_matrix_ordered['sum'] = noncoding_conserved_matrix_ordered.select_dtypes(include='number').sum(axis=1)

# Sort by the sum column to order by the number of species present
noncoding_conserved_matrix_ordered = noncoding_conserved_matrix_ordered.sort_values(by="sum", ascending=False)
print(len(set(noncoding_conserved_matrix_ordered.orfID.values.tolist())))

# Display the result
noncoding_conserved_matrix_ordered =  noncoding_conserved_matrix_ordered.merge(noncoding_conserved[['gene_name','gene_type','orfID']], on="orfID")
noncoding_conserved_matrix_ordered.drop_duplicates(inplace=True)

noncoding_conserved_matrix_ordered.to_csv(os.path.join(evoDir,"Clustering/noncodingHuman_Conserved_matrix.tsv"), sep="\t", index=None)
print(len(set(noncoding_conserved_matrix_ordered.orfID.values.tolist())))

# Display the result
noncoding_conserved_matrix_ordered =  noncoding_conserved_matrix_ordered.merge(noncoding_conserved[['gene_name','gene_type','orfID']], on="orfID")
noncoding_conserved_matrix_ordered.drop_duplicates(inplace=True)

noncoding_conserved_matrix_ordered.to_csv(os.path.join(evoDir,"Clustering/noncodingHuman_Conserved_matrix.tsv"), sep="\t", index=None)
print(len(set(noncoding_conserved_matrix_ordered.orfID.values.tolist())))


176
176
176
176
176


/tmp/ipykernel_2049236/3235198060.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noncoding_conserved_to_matrix['presence'] = 1
/tmp/ipykernel_2049236/3235198060.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noncoding_conserved_matrix_ordered['sum'] = noncoding_conserved_matrix_ordered.select_dtypes(include='number').sum(axis=1)


In [17]:
### all noncoding
noncoding_conserved_touse = noncoding_conserved
noncoding_conserved_touse['specie'] = noncoding_conserved_touse[1].str.split("_", expand=True)[0]
# noncoding_NonConserved.drop("specie",axis=1, inplace=True)
noncoding_to_matrix_raw = pd.concat([noncoding_conserved, noncoding_NonConserved])
noncoding_to_matrix = noncoding_to_matrix_raw[['orfID', 'specie']]
noncoding_to_matrix = noncoding_to_matrix.fillna('human').drop_duplicates()
noncoding_to_matrix['presence'] = 1
print(noncoding_to_matrix.groupby("specie").count())
noncoding_to_matrix

          orfID  presence
specie                   
chicken      38        38
human      3308      3308
macaca      111       111
mouse        92        92
opossum      61        61
platypus     51        51


,orfID,specie,presence
211,ENST00000581862.7:18:-|12|3689:297:567|noncodi...,macaca,1
286,ENST00000636308.1:7:+|49|4646:635:1763|noncodi...,macaca,1
359,ENST00000701281.2:7:+|2|1243:33:507|noncoding|ATG,macaca,1
360,ENST00000701281.2:7:+|2|1243:33:507|noncoding|ATG,mouse,1
642,ENST00000683452.1:3:+|19|3753:358:1843|noncodi...,macaca,1
...,...,...,...
26360,ENST00000666621.1:10:-|25|3011:473:665|noncodi...,human,1
26361,ENST00000671403.1:2:+|11|4190:235:391|noncodin...,human,1
26372,ENST00000686765.2:3:+|2|1688:28:175|noncoding|ATG,human,1
26373,ENST00000690653.3:2:+|23|2292:494:677|noncodin...,human,1


In [18]:
# Pivoting the data without creating an 'id', and keeping 'orfID' as the unique identifier
noncoding_matrix = noncoding_to_matrix.pivot_table(index='orfID', columns='specie', values='presence', aggfunc='max')
# Fill NaN values with 0
noncoding_matrix = noncoding_matrix.fillna(0)
# Reset index so 'orfID' becomes a column instead of an index
noncoding_matrix = noncoding_matrix.reset_index()

# Rename columns as necessary
noncoding_matrix.columns = ["orfID", "chicken", "human", "macaca", "mouse", "opossum", "platypus"]
print(len(noncoding_matrix))
# Reorder columns for the final matrix
noncoding_matrix_ordered = noncoding_matrix[["orfID", "human","macaca", "mouse", "opossum", "platypus", "chicken"]]

# # Calculate sum across species columns to identify how many species are present
noncoding_matrix_ordered['sum'] = noncoding_matrix_ordered.select_dtypes(include='number').sum(axis=1)

# Sort by the sum column to order by the number of species present
noncoding_matrix_ordered = noncoding_matrix_ordered.sort_values(by="sum", ascending=False)

# Display the result
noncoding_matrix_ordered = noncoding_matrix_ordered.merge(noncoding_to_matrix_raw[['gene_name','gene_type','orfID']], on="orfID")
noncoding_matrix_ordered.drop_duplicates(inplace=True)

noncoding_matrix_ordered.to_csv(os.path.join(evoDir,"Clustering/noncodingHuman_matrix.tsv"), sep="\t", index=None)
print(len(set(noncoding_matrix_ordered.orfID.values.tolist())))
noncoding_matrix_ordered

3308
3308


/tmp/ipykernel_2049236/1563675081.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noncoding_matrix_ordered['sum'] = noncoding_matrix_ordered.select_dtypes(include='number').sum(axis=1)


,orfID,human,macaca,mouse,opossum,platypus,chicken,sum,gene_name,gene_type
0,ENST00000456500.1:2:+|1|1047:1:1048|noncoding|ATG,1.0,1.0,1.0,1.0,1.0,1.0,6.0,ENSG00000225402,processed_pseudogene
7,ENST00000429606.1:5:-|1|459:1:412|noncoding|ATG,1.0,1.0,1.0,1.0,1.0,1.0,6.0,ENSG00000229738,processed_pseudogene
14,ENST00000397254.2:X:+|26|1590:721:1591|noncodi...,1.0,1.0,1.0,1.0,1.0,1.0,6.0,RBM39P1,processed_pseudogene
21,ENST00000439215.1:1:+|10|681:226:682|noncoding...,1.0,1.0,1.0,1.0,1.0,1.0,6.0,ATP6V1E1P1,processed_pseudogene
31,ENST00000398050.3:10:-|1|1076:1:538|noncoding|ATG,1.0,1.0,1.0,1.0,1.0,1.0,6.0,ALDOAP2,processed_pseudogene
...,...,...,...,...,...,...,...,...,...,...
4000,ENST00000510731.1:5:-|38|1099:564:885|noncodin...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,ENSG00000249541,processed_pseudogene
4001,ENST00000510896.1:5:-|42|1379:722:1361|noncodi...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,HNRNPKP1,processed_pseudogene
4002,ENST00000510897.7:8:+|162|3536:3164:3212|nonco...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,LINC00964,lncRNA
4003,ENST00000510897.7:8:+|1|3536:20:77|noncoding|CTG,1.0,0.0,0.0,0.0,0.0,0.0,1.0,LINC00964,lncRNA


In [19]:

# Pivoting the data without creating an 'id', and keeping 'orfID' as the unique identifier
noncoding_matrix = noncoding_to_matrix.pivot_table(index='orfID', columns='specie', values='presence', aggfunc='max')
noncoding_matrix
# Fill NaN values with 0
noncoding_matrix = noncoding_matrix.fillna(0)

# Reset index so 'orfID' becomes a column instead of an index
noncoding_matrix = noncoding_matrix.reset_index()

# Rename columns as necessary
noncoding_matrix.columns = ["orfID", "chicken", "human", "macaca", "mouse", "opossum", "platypus"]

# Reorder columns for the final matrix
noncoding_matrix_ordered = noncoding_matrix[["orfID", "human","macaca", "mouse", "opossum", "platypus", "chicken"]]

# Calculate sum across species columns to identify how many species are present
noncoding_matrix_ordered['sum'] = noncoding_matrix_ordered.select_dtypes(include='number').sum(axis=1)

# Sort by the sum column to order by the number of species present
noncoding_matrix_ordered = noncoding_matrix_ordered.sort_values(by="sum", ascending=False)

# Display the result
noncoding_matrix_ordered =  noncoding_matrix_ordered.merge(noncoding_to_matrix_raw[['gene_name','gene_type','orfID']], on="orfID")
noncoding_matrix_ordered.drop_duplicates(inplace=True)

noncoding_matrix_ordered.to_csv(os.path.join(evoDir,"Clustering/noncodingHuman_matrix.tsv"), sep="\t", index=None)
print(len(set(noncoding_matrix_ordered.orfID.values.tolist())))

/tmp/ipykernel_2049236/3967721755.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noncoding_matrix_ordered['sum'] = noncoding_matrix_ordered.select_dtypes(include='number').sum(axis=1)


3308


### alternative ORFs

In [20]:
# if column 1 starts with a specie, it is conserved
altORFs_conserved = altORFs[~altORFs[1].str.startswith("ENST")]
altORFs_conserved.to_csv(os.path.join(evoDir,"Clustering/altORFsHuman_Conserved.tsv"), header=None, index=None, sep="\t")
print("Conserved altORFs human ORFs")
print(len(set(altORFs_conserved[0].values.tolist())))
altORFs_conserved = altORFs_conserved[['gene_name','gene_type',0,1]]
altORFs_conserved.columns = ['gene_name','gene_type','orfID',1]

# if column 1 starts with a human gene, it is not conserved
altORFs_NonConserved = altORFs[altORFs[1].str.startswith("ENST")]
altORFs_NonConserved.to_csv(os.path.join(evoDir,"Clustering/altORFsHuman_NonConserved.tsv"), header=None, index=None, sep="\t")
print("Not-conserved altORFs human ORFs")
print(len(set(altORFs_NonConserved[0].values.tolist())))
altORFs_NonConserved = altORFs_NonConserved[['gene_name','gene_type',0,1]]
altORFs_NonConserved.columns = ['gene_name','gene_type','orfID',1]


Conserved altORFs human ORFs
92
Not-conserved altORFs human ORFs
7516


In [21]:
## conserved altORFs
altORFs_conserved_to_matrix = altORFs_conserved
print(len(set(altORFs_conserved_to_matrix.orfID.values.tolist())))
# Assuming noncoding_conserved_to_matrix is already defined
altORFs_conserved_to_matrix['specie'] = altORFs_conserved_to_matrix[1].str.split("_", expand=True)[0]
altORFs_conserved_to_matrix = altORFs_conserved_to_matrix[['orfID', 'specie']]
altORFs_conserved_to_matrix.groupby('specie').count()
altORFs_conserved_to_matrix['presence'] = 1
## control
print(len(set(altORFs_conserved_to_matrix.orfID.values.tolist())))

# Pivoting the data without creating an 'id', and keeping 'orfID' as the unique identifier
altORFs_conserved_matrix = altORFs_conserved_to_matrix.pivot_table(index='orfID', columns='specie', values='presence', aggfunc='max')

# Fill NaN values with 0
altORFs_conserved_matrix = altORFs_conserved_matrix.fillna(0)

# Reset index so 'orfID' becomes a column instead of an index
altORFs_conserved_matrix = altORFs_conserved_matrix.reset_index()

# Reorder columns for the final matrix
altORFs_conserved_matrix_ordered = altORFs_conserved_matrix[["orfID", "macaca", "mouse", "opossum", "platypus", "chicken"]]

# Calculate sum across species columns to identify how many species are present
altORFs_conserved_matrix_ordered['sum'] = altORFs_conserved_matrix_ordered.select_dtypes(include='number').sum(axis=1)

# Sort by the sum column to order by the number of species present
altORFs_conserved_matrix_ordered = altORFs_conserved_matrix_ordered.sort_values(by="sum", ascending=False)
print(len(set(altORFs_conserved_matrix_ordered.orfID.values.tolist())))

# Display the result
altORFs_conserved_matrix_ordered =  altORFs_conserved_matrix_ordered.merge(altORFs_conserved[['gene_name','gene_type','orfID']], on="orfID")
altORFs_conserved_matrix_ordered.drop_duplicates(inplace=True)

altORFs_conserved_matrix_ordered.to_csv(os.path.join(evoDir,"Clustering/altORFsHuman_Conserved_matrix.tsv"), sep="\t", index=None)
print(len(set(altORFs_conserved_matrix_ordered.orfID.values.tolist())))

# Display the result
altORFs_conserved_matrix_ordered =  altORFs_conserved_matrix_ordered.merge(altORFs_conserved[['gene_name','gene_type','orfID']], on="orfID")
altORFs_conserved_matrix_ordered.drop_duplicates(inplace=True)

altORFs_conserved_matrix_ordered.to_csv(os.path.join(evoDir,"Clustering/altORFsHuman_Conserved_matrix.tsv"), sep="\t", index=None)
print(len(set(altORFs_conserved_matrix_ordered.orfID.values.tolist())))


92
92
92
92
92


/tmp/ipykernel_2049236/2466750026.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  altORFs_conserved_to_matrix['presence'] = 1


In [22]:
### all altORFs
altORFs_conserved_touse = altORFs_conserved
altORFs_conserved_touse['specie'] = altORFs_conserved_touse[1].str.split("_", expand=True)[0]
altORFs_to_matrix_raw = pd.concat([altORFs_conserved, altORFs_NonConserved])
altORFs_to_matrix = altORFs_to_matrix_raw[['orfID', 'specie']]
altORFs_to_matrix = altORFs_to_matrix.fillna('human').drop_duplicates()
altORFs_to_matrix['presence'] = 1
altORFs_to_matrix = altORFs_to_matrix.replace("TCONS", "human")
print(altORFs_to_matrix.groupby("specie").count())


          orfID  presence
specie                   
chicken       6         6
human      7518      7518
macaca       44        44
mouse        37        37
opossum      20        20
platypus      9         9


In [23]:
# Pivoting the data without creating an 'id', and keeping 'orfID' as the unique identifier
altORFs_matrix = altORFs_to_matrix.pivot_table(index='orfID', columns='specie', values='presence', aggfunc='max')
# Fill NaN values with 0
altORFs_matrix = altORFs_matrix.fillna(0)
# Reset index so 'orfID' becomes a column instead of an index
altORFs_matrix = altORFs_matrix.reset_index()
altORFs_matrix
# Rename columns as necessary
altORFs_matrix.columns = ["orfID", "chicken", "human", "macaca", "mouse", "opossum", "platypus"]
print(len(altORFs_matrix))
# Reorder columns for the final matrix
altORFs_matrix_ordered = altORFs_matrix[["orfID", "human","macaca", "mouse", "opossum", "platypus", "chicken"]]

# # Calculate sum across species columns to identify how many species are present
altORFs_matrix_ordered['sum'] = altORFs_matrix_ordered.select_dtypes(include='number').sum(axis=1)

# Sort by the sum column to order by the number of species present
altORFs_matrix_ordered = altORFs_matrix_ordered.sort_values(by="sum", ascending=False)

# Display the result
altORFs_matrix_ordered = altORFs_matrix_ordered.merge(altORFs_to_matrix_raw[['gene_name','gene_type','orfID']], on="orfID")
altORFs_matrix_ordered.drop_duplicates(inplace=True)

altORFs_matrix_ordered.to_csv(os.path.join(evoDir,"Clustering/altORFsHuman_matrix.tsv"), sep="\t", index=None)
print(len(set(altORFs_matrix_ordered.orfID.values.tolist())))
altORFs_matrix_ordered

7516
7516


/tmp/ipykernel_2049236/70459802.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  altORFs_matrix_ordered['sum'] = altORFs_matrix_ordered.select_dtypes(include='number').sum(axis=1)


,orfID,human,macaca,mouse,opossum,platypus,chicken,sum,gene_name,gene_type
0,ENST00000532219.5:5:+|456|8246:7706:7940|odORF...,1.0,1.0,1.0,1.0,1.0,1.0,6.0,ANKHD1-EIF4EBP3,protein_coding
13,ENST00000618666.4:12:-|46|6915:1015:2341|dORF|ATG,1.0,1.0,1.0,1.0,1.0,1.0,6.0,CCNT1,protein_coding
19,ENST00000695752.1:1:+|41|1548:541:1222|odORF|ATG,1.0,1.0,1.0,1.0,0.0,1.0,5.0,C1QC,protein_coding
24,ENST00000628377.2:3:-|53|2214:875:2054|dORF|ATG,1.0,0.0,1.0,1.0,1.0,1.0,5.0,HACL1,protein_coding
29,ENST00000643349.2:11:-|42|2861:727:1270|dORF|ATG,1.0,1.0,1.0,0.0,1.0,1.0,5.0,ENSG00000284779,protein_coding
...,...,...,...,...,...,...,...,...,...,...
7822,ENST00000339697.5:6:-|159|3125:2425:2530|dORF|CTG,1.0,0.0,0.0,0.0,0.0,0.0,1.0,SERINC1,protein_coding
7823,ENST00000339656.8:19:+|3|6885:22:76|uORF|ATG,1.0,0.0,0.0,0.0,0.0,0.0,1.0,ZNF587,protein_coding
7824,ENST00000339656.8:19:+|113|6885:1827:2277|odOR...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,ZNF587,protein_coding
7825,ENST00000339618.8:17:+|109|3828:1745:1805|odOR...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,ALDH3A2,protein_coding



### CANONICAL

In [24]:
# if column 1 starts with a specie, it is conserved
canonical_conserved = canonical[~canonical[1].str.startswith("ENST")]
canonical_conserved.to_csv(os.path.join(evoDir,"Clustering/canonicalHuman_Conserved.tsv"), header=None, index=None, sep="\t")
print("Conserved canonical human ORFs")
print(len(set(canonical_conserved[0].values.tolist())))
canonical_conserved = canonical_conserved[['gene_name','gene_type',0,1]]
canonical_conserved.columns = ['gene_name','gene_type','orfID',1]

# if column 1 starts with a human gene, it is not conserved
canonical_NonConserved = canonical[canonical[1].str.startswith("ENST")]
canonical_NonConserved.to_csv(os.path.join(evoDir,"Clustering/canonicalHuman_NonConserved.tsv"), header=None, index=None, sep="\t")
print("AltORFs noncoding human ORFs")
print(len(set(canonical_NonConserved[0].values.tolist())))
canonical_NonConserved = canonical_NonConserved[['gene_name','gene_type',0,1]]
canonical_NonConserved.columns = ['gene_name','gene_type','orfID',1]


Conserved canonical human ORFs
2594
AltORFs noncoding human ORFs
2909


In [25]:
## conserved canonical
canonical_conserved_to_matrix = canonical_conserved
print(len(set(canonical_conserved_to_matrix.orfID.values.tolist())))
# Assuming noncoding_conserved_to_matrix is already defined
canonical_conserved_to_matrix['specie'] = canonical_conserved_to_matrix[1].str.split("_", expand=True)[0]
canonical_conserved_to_matrix = canonical_conserved_to_matrix[['orfID', 'specie']]
canonical_conserved_to_matrix.groupby('specie').count()
canonical_conserved_to_matrix['presence'] = 1
## control
print(len(set(canonical_conserved_to_matrix.orfID.values.tolist())))

# Pivoting the data without creating an 'id', and keeping 'orfID' as the unique identifier
canonical_conserved_matrix = canonical_conserved_to_matrix.pivot_table(index='orfID', columns='specie', values='presence', aggfunc='max')

# Fill NaN values with 0
canonical_conserved_matrix = canonical_conserved_matrix.fillna(0)

# Reset index so 'orfID' becomes a column instead of an index
canonical_conserved_matrix = canonical_conserved_matrix.reset_index()

# Reorder columns for the final matrix
canonical_conserved_matrix_ordered = canonical_conserved_matrix[["orfID", "macaca", "mouse", "opossum", "platypus", "chicken"]]

# Calculate sum across species columns to identify how many species are present
canonical_conserved_matrix_ordered['sum'] = canonical_conserved_matrix_ordered.select_dtypes(include='number').sum(axis=1)

# Sort by the sum column to order by the number of species present
canonical_conserved_matrix_ordered = canonical_conserved_matrix_ordered.sort_values(by="sum", ascending=False)
print(len(set(canonical_conserved_matrix_ordered.orfID.values.tolist())))

# Display the result
canonical_conserved_matrix_ordered =  canonical_conserved_matrix_ordered.merge(canonical_conserved[['gene_name','gene_type','orfID']], on="orfID")
canonical_conserved_matrix_ordered.drop_duplicates(inplace=True)

canonical_conserved_matrix_ordered.to_csv(os.path.join(evoDir,"Clustering/canonicalHuman_Conserved_matrix.tsv"), sep="\t", index=None)
print(len(set(canonical_conserved_matrix_ordered.orfID.values.tolist())))

# Display the result
canonical_conserved_matrix_ordered =  canonical_conserved_matrix_ordered.merge(canonical_conserved[['gene_name','gene_type','orfID']], on="orfID")
canonical_conserved_matrix_ordered.drop_duplicates(inplace=True)

canonical_conserved_matrix_ordered.to_csv(os.path.join(evoDir,"Clustering/canonicalHuman_Conserved_matrix.tsv"), sep="\t", index=None)
print(len(set(canonical_conserved_matrix_ordered.orfID.values.tolist())))


2594
2594
2594
2594
2594


/tmp/ipykernel_2049236/3948074870.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  canonical_conserved_to_matrix['presence'] = 1
/tmp/ipykernel_2049236/3948074870.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  canonical_conserved_matrix_ordered['sum'] = canonical_conserved_matrix_ordered.select_dtypes(include='number').sum(axis=1)


In [26]:
### all canonical
canonical_conserved_touse = canonical_conserved
canonical_conserved_touse['specie'] = canonical_conserved_touse[1].str.split("_", expand=True)[0]
canonical_to_matrix_raw = pd.concat([canonical_conserved, canonical_NonConserved])
canonical_to_matrix = canonical_to_matrix_raw[['orfID', 'specie']]
canonical_to_matrix = canonical_to_matrix.fillna('human').drop_duplicates()
canonical_to_matrix['presence'] = 1
canonical_to_matrix = canonical_to_matrix.replace("TCONS", "human")
print(canonical_to_matrix.groupby("specie").count())


          orfID  presence
specie                   
chicken    1466      1466
human      2909      2909
macaca     2320      2320
mouse      1989      1989
opossum    1629      1629
platypus   1465      1465


In [27]:
# Pivoting the data without creating an 'id', and keeping 'orfID' as the unique identifier
canonical_matrix = canonical_to_matrix.pivot_table(index='orfID', columns='specie', values='presence', aggfunc='max')
# Fill NaN values with 0
canonical_matrix = canonical_matrix.fillna(0)
# Reset index so 'orfID' becomes a column instead of an index
canonical_matrix = canonical_matrix.reset_index()
canonical_matrix
# Rename columns as necessary
canonical_matrix.columns = ["orfID", "chicken", "human", "macaca", "mouse", "opossum", "platypus"]
print(len(canonical_matrix))
# Reorder columns for the final matrix
canonical_matrix_ordered = canonical_matrix[["orfID", "human","macaca", "mouse", "opossum", "platypus", "chicken"]]

# # Calculate sum across species columns to identify how many species are present
canonical_matrix_ordered['sum'] = canonical_matrix_ordered.select_dtypes(include='number').sum(axis=1)

# Sort by the sum column to order by the number of species present
canonical_matrix_ordered = canonical_matrix_ordered.sort_values(by="sum", ascending=False)

# Display the result
canonical_matrix_ordered = canonical_matrix_ordered.merge(canonical_to_matrix_raw[['gene_name','gene_type','orfID']], on="orfID")
canonical_matrix_ordered.drop_duplicates(inplace=True)

canonical_matrix_ordered.to_csv(os.path.join(evoDir,"Clustering/canonicalHuman_matrix.tsv"), sep="\t", index=None)
print(len(set(canonical_matrix_ordered.orfID.values.tolist())))
canonical_matrix_ordered

2909


/tmp/ipykernel_2049236/1133186206.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  canonical_matrix_ordered['sum'] = canonical_matrix_ordered.select_dtypes(include='number').sum(axis=1)


2909


,orfID,human,macaca,mouse,opossum,platypus,chicken,sum,gene_name,gene_type
0,ENST00000850822.1:Y:-|1|3290:12:1563|canonical...,1.0,1.0,1.0,1.0,1.0,1.0,6.0,GTPBP6,protein_coding
6,ENST00000398805.8:9:+|4|3174:57:1314|canonical...,1.0,1.0,1.0,1.0,1.0,1.0,6.0,HSDL2,protein_coding
12,ENST00000400590.8:13:-|5|5355:133:1231|canonic...,1.0,1.0,1.0,1.0,1.0,1.0,6.0,ZDHHC20,protein_coding
29,ENST00000301328.9:17:-|3|1809:25:967|canonical...,1.0,1.0,1.0,1.0,1.0,1.0,6.0,GLOD4,protein_coding
35,ENST00000400405.4:13:-|6|7075:125:935|canonica...,1.0,1.0,1.0,1.0,1.0,1.0,6.0,SIAH3,protein_coding
...,...,...,...,...,...,...,...,...,...,...
14535,ENST00000557436.2:14:+|10|2956:277:589|canonic...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,PTGER2,protein_coding
14536,ENST00000553909.1:14:+|9|1498:191:287|canonica...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,ENSG00000259171,protein_coding
14537,ENST00000370629.7:X:+|8|1852:92:878|canonical|ATG,1.0,0.0,0.0,0.0,0.0,0.0,1.0,CD40LG,protein_coding
14538,ENST00000552878.5:12:-|7|1052:84:633|canonical...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,FKBP11,protein_coding


Find the oldest specie containg the ORF translated

In [28]:
species_ordered = ['chicken','platypus','opossum','mouse','macaca','human']
def find_last_species(row):
    for sp in species_ordered:
        if row[sp] == 1.0:  # Check if the value is 1.0
            return sp
    return None  # Return None if no species has a 1

In [29]:
# Apply the function to each row to create a new column 'oldest_species'
canonical_matrix_ordered['oldest_species'] = canonical_matrix_ordered.apply(find_last_species, axis=1)
canonical_matrix_ordered = species_and_years.merge(canonical_matrix_ordered, left_on=["specie"], right_on="oldest_species")
canonical_matrix_ordered[['orfID','gene_name','gene_type','oldest_species','age']].to_csv(os.path.join(evoDir,"Clustering/canonicalHuman_matrix_oldestAge.tsv"), sep="\t", index=None)

noncoding_matrix_ordered['oldest_species'] = noncoding_matrix_ordered.apply(find_last_species, axis=1)
noncoding_matrix_ordered = species_and_years.merge(noncoding_matrix_ordered, left_on=["specie"], right_on="oldest_species")
noncoding_matrix_ordered[['orfID','gene_name','gene_type','oldest_species','age']].to_csv(os.path.join(evoDir,"Clustering/noncodingHuman_matrix_oldestAge.tsv"), sep="\t", index=None)


altORFs_matrix_ordered['oldest_species'] = altORFs_matrix_ordered.apply(find_last_species, axis=1)
altORFs_matrix_ordered = species_and_years.merge(altORFs_matrix_ordered, left_on=["specie"], right_on="oldest_species")
altORFs_matrix_ordered[['orfID','gene_name','gene_type','oldest_species','age']].to_csv(os.path.join(evoDir,"Clustering/altORFsHuman_matrix_oldestAge.tsv"), sep="\t", index=None)


complete_matrix_age = pd.concat([noncoding_matrix_ordered[['orfID','gene_name','gene_type','oldest_species','age']],altORFs_matrix_ordered[['orfID','gene_name','gene_type','oldest_species','age']], canonical_matrix_ordered[['orfID','gene_name','gene_type','oldest_species','age']]])
complete_matrix_age.to_csv(os.path.join(evoDir,"Clustering/allHuman_matrix_oldestAge.tsv"), sep="\t", index=None)
